# PROVIAMO A FARE UNA SENTIMENT ANALYSIS

In [1]:
#impostiamo la directory di lavoro
import os
os.getcwd()

os.chdir('C:\\Users\\Utente\\Desktop\\sentimentale\\aclImdb') #C:\Users\Utente\Desktop\sentimentale
print(os.getcwd())
#nell'affrontare questa sentiment analysis noi partiamo direttamente dal testo delle cartelle pos e neg (praticamente rappresentano il dataset coi relativi tag: pos e neg)

C:\Users\Utente\Desktop\sentimentale\aclImdb


In [2]:
#IN QUESTA CELLA ANDIAMO A IMPORTARE LE RECENSIONI CON I RELATIVI "VOTI" (pos, neg)

#files_path= 'C:\\Users\\Utente\\Desktop\\sentimentale\\aclImdb'

#reviews=[] #testo della recensione
#y=[] #commento della recensione (pos,neg)

#considerando che i file txt con le rensioni si trovano in sottocartelle chiamate "pos" o "neg", quando prendo i file delle prime gli do valore 1 altrimenti 0
labels=["pos", "neg"]


label_map={"pos":1, "neg":0}

from os import listdir
import pandas as pd

reviews_train=[]
y_train=[]

files_path= "C:\\Users\\Utente\\Desktop\\sentimentale\\aclImdb\\train" #path del train
labels=["pos","neg"]
for label in labels:
    path=files_path+"/"+label
    for file in listdir(path): #listdir è un metodo che ci permette di ottenere la lista di file dentro al path
        review_file=open(path+"/"+file, encoding="utf-8")
        review=review_file.read()
        review_file.close()

        reviews_train.append(review) #features
        y_train.append(label_map[label]) #target

        
reviews_test=[]
y_test=[]        
files_path= "C:\\Users\\Utente\\Desktop\\sentimentale\\aclImdb\\test" #path del train
labels=["pos","neg"]
for label in labels:
    path=files_path+"/"+label
    for file in listdir(path): #listdir è un metodo che ci permette di ottenere la lista di file dentro al path
        review_file=open(path+"/"+file, encoding="utf-8")
        review=review_file.read()
        review_file.close()

        reviews_test.append(review) #features
        y_test.append(label_map[label]) #target
        
#reviews_train, y_train=load_xy("C:\\Users\\Utente\\Desktop\\sentimentale\\aclImdb\\train")
#reviews_test, y_test=load_xy("C:\\Users\\Utente\\Desktop\\sentimentale\\aclImdb\\test")


In [3]:
#ENTRIAMO NEL CUORE DEL PROCESSO DI MACHINE LEARNING
#dato che vogliamo fare tutto da soli.... creaiamoci un modello bag of words, senza usare modelli preprocessati presi dal web

from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer #questo ci serve per effettuare la bag of words

#bow=bag of words. Ci serve per trafsormare le stringhe in formato numerico
bow= CountVectorizer(max_features=5000) #stabiliamo quante parole vogliamo prendere

#sklearn ha 2 classi a disposizioni: i trasformatori e gli estimatori 
#trasformatori = che servono a traformare i dati
#gli estimatori = che servono a costruire i modelli. Negli estimatori troviamo 2 metodi: fit (che realizza i calcoli per effettuare la trasformazione)  e transform (che effettua fisicamente la traformazione)
bow_train=bow.fit_transform(reviews_train) #abbiamo traformato le stringhe in bag of words, e successivamente le trasformiamo nell'array di train
X_train = bow_train.toarray()

bow_test = bow.transform(reviews_test) #essendo la parte del dataset di test... non dobbiamo fare il "fit" ma solo transform
X_test = bow_test.toarray()


#standardizziamo i dati di test e train... perchè potrebbero avere valori differenti tra loro (1000000, 0.00001)
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

X_train=ss.fit_transform(X_train)
X_test = ss.transform(X_test)

#se la standardizzazione è avvenuta correttamente, la media dovrebbe essere molto vicina allo zero e la deviazione standard molto vicina a 1
X_train.mean()
X_train.std()
X_test.mean()
X_test.std()

0.9925719860299927

In [11]:
#finalmente effettuaiamo l'addestramento del modello... dato che la risposta è un "SI"/"NO"(pos, neg) puntiamo, ovviamente, alla logit
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(C=0.001) #impostiamo questo valore (che non è obbligatorio) per evitare l'overfitting

lr.fit(X_train, y_train) #feature, target

C:\Users\Utente\Anaconda3.1\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
#andiamo ad effettuar dei calcoli sul modello per vedere se è buono oppure no
from sklearn.metrics import log_loss, accuracy_score #la prima è la funzione di costo e la 2a è l'accuratezza del modello

train_pred=lr.predict(X_train) #calcoliamo e valutiamo le predizioni del modello sul set di addestramento
train_pred_proba=lr.predict_proba(X_train) #prbabilità che un esempio all'interno del set di addestramento alla classe positiva (???)
accuracy_score(y_train, train_pred) #percentuale di classificazioni corrette. Ad esempio, per un 0.8 vuol dire che il nostro modello ha classficato bene l'80% degli esempi
log_loss(y_train, train_pred_proba) #più è bassa meglio è. Vuol dire che il modello è sicuro sulle predizioni effettuate

train_pred=lr.predict(X_test) #calcoliamo e valutiamo le predizioni del modello sul set di addestramento
train_pred_proba=lr.predict_proba(X_test) #prbabilità che un esempio all'interno del set di addestramento alla classe positiva (???)
accuracy_score(y_test, train_pred) #percentuale di classificazioni corrette. Ad esempio, per un 0.8 vuol dire che il nostro modello ha classficato bene l'80% degli esempi
log_loss(y_test, train_pred_proba) #più è bassa meglio è. Vuol dire che il modello è sicuro sulle predizioni effettuate


0.3124911583591352

In [18]:
#finalmente proviamo le recensioni
review="I hate this film"
x=bow.transform([review]) #trasformiamo la string attraverso il bag of word, e poi effettuiamo (solo) la transform, perchè è come se fosse un training set
lr.predict(x)

array([0])